In [1]:

import jax
import jax.numpy as jnp
from sklearn.linear_model import LogisticRegression

from experiments.logisticRegression.mnist.load_mnist import mnist_dataset

OUTPUT_PATH = "./output_mean_field"
OP_key = jax.random.PRNGKey(0)
logreg = LogisticRegression(penalty=None, fit_intercept=False)
jax.config.update("jax_enable_x64", True)
mnist_train, mnist_test = mnist_dataset(return_test=True, flip=False)
flipped_predictors, response = mnist_train
flipped_predictors_test, response_test = mnist_test

In [2]:
logreg.fit(flipped_predictors, response)

LogisticRegression(fit_intercept=False, penalty=None)

In [5]:
test_prediction = logreg.predict(flipped_predictors_test)

In [9]:
jnp.sum(test_prediction == response_test) / len(response_test)

Array(0.98925281, dtype=float64)

In [11]:
train_prediction = logreg.predict(flipped_predictors)

In [13]:
jnp.sum(train_prediction == response) / len(response)

Array(0.99991507, dtype=float64)

In [15]:
from sklearn.metrics import log_loss

log_loss(response_test, logreg.predict_proba(flipped_predictors_test))
log_loss(response, logreg.predict_proba(flipped_predictors))

0.001010022643566054